# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

2022-07-16 14:39:30.987104: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
cv_seeds = [13, 29, 42, 55, 73]

## Load data

In [5]:
data_name = 'adult'

In [6]:
x, y, a = load_data(data_name)
raw_data = (x, y, a)

In [7]:
xdim = x.shape[1]
ydim = y.shape[1]
adim = a.shape[1]
zdim = 8

## Result file

In [8]:
header = "model_name", "cv_seed","fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [9]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

2022-07-16 14:39:36.922619: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-16 14:39:36.923289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-16 14:39:36.964135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 14:39:36.964636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2022-07-16 14:39:36.964673: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-16 14:39:36.966441: I tensorflow/stream_executor/platform/def

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc


2022-07-16 14:39:37.962711: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-07-16 14:39:38.837381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


> 1 | -0.04290948435664177 | 0.42633676528930664 | 0.5550652146339417 | 0.9192766547203064 | 0.7139423076923077 | 0.6182312753036437 | 0.027201417004048583
> 2 | -0.06996754556894302 | 0.40373629331588745 | 0.6136389374732971 | 0.9192766547203064 | 0.8263853744939271 | 0.6638410931174089 | 0.027201417004048583


KeyboardInterrupt: 

### LAFTR for Eq Odds

In [13]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.015396754257380962 | 0.5530157089233398 | 0.5068254470825195 | 0.9201741218566895 | 0.6868368700265252 | 0.622886273209549 | -101.54302055702918
> 2 | 0.008753667585551739 | 0.5532335042953491 | 0.5269724726676941 | 0.9201741218566895 | 0.8193385278514589 | 0.6635444297082228 | -101.54302055702918
> 3 | 0.005823870655149221 | 0.543075442314148 | 0.5256038308143616 | 0.9201741218566895 | 0.8318136604774535 | 0.6675232095490716 | -101.54302055702918
> 4 | 0.0027174155693501234 | 0.5328130722045898 | 0.5246608257293701 | 0.9201741218566895 | 0.8345490716180372 | 0.6670258620689655 | -101.54302055702918
> 5 | 0.00010138750076293945 | 0.5245352387428284 | 0.5242310762405396 | 0.9201741218566895 | 0.8374502652519894 | 0.6672330901856764 | -101.54302055702918
> 6 | -0.0005825459957122803 | 0.5223731994628906 | 0.5241208076477051 | 0.9201741218566895 | 0.8391909814323607 | 0.6674817639257294 | -101.

### LAFTR for Eq Opp

In [14]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(
            xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0058588190004229546 | 0.5432302355766296 | 0.5288085341453552 | 0.9201741218566895 | 0.6884946949602122 | 0.5864555702917772 | -101.54302055702918
> 2 | -0.0014059869572520256 | 0.548180103302002 | 0.551641047000885 | 0.9201741218566895 | 0.8240218832891246 | 0.662425397877984 | -101.54302055702918
> 3 | -0.005373783875256777 | 0.5400211215019226 | 0.5532488822937012 | 0.9201741218566895 | 0.8269645225464191 | 0.667606100795756 | -101.54302055702918
> 4 | -0.009550941176712513 | 0.5298053026199341 | 0.5533152222633362 | 0.9201741218566895 | 0.8313577586206896 | 0.6671916445623343 | -101.54302055702918
> 5 | -0.012470726855099201 | 0.5226964950561523 | 0.5533937215805054 | 0.9201741218566895 | 0.8344661803713528 | 0.6675646551724138 | -101.54302055702918
> 6 | -0.014044043608009815 | 0.5188814401626587 | 0.553451418876648 | 0.9201741218566895 | 0.8355852122015915 | 0.666901525198939 | -101.54

## Saving into DF then CSV

In [15]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,0.2,0.850233,0.818917,0.942081,0.960377,0.834281,0.893804,0.901955,1703.0,55.0,82.0,119.0,2499.0,301.0,463.0,794.0
1,LAFTR4DP,0.5,0.850399,0.832913,0.965804,0.997981,0.841565,0.904435,0.918298,1697.0,61.0,77.0,124.0,2517.0,283.0,479.0,778.0
2,LAFTR4DP,0.7,0.848238,0.855747,0.954423,0.959451,0.851976,0.898203,0.900423,1690.0,68.0,75.0,126.0,2550.0,250.0,520.0,737.0
3,LAFTR4DP,1.0,0.846742,0.870077,0.951554,0.940750,0.858251,0.896094,0.891274,1676.0,82.0,73.0,128.0,2564.0,236.0,531.0,726.0
4,LAFTR4EqOdds,0.2,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
5,LAFTR4EqOdds,0.5,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
6,LAFTR4EqOdds,0.7,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
7,LAFTR4EqOdds,1.0,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
8,LAFTR4EqOpp,0.2,0.845911,0.864881,0.949424,0.939942,0.855291,0.894683,0.890451,1667.0,91.0,69.0,132.0,2540.0,260.0,507.0,750.0
9,LAFTR4EqOpp,0.5,0.845911,0.864881,0.949424,0.939942,0.855291,0.894683,0.890451,1667.0,91.0,69.0,132.0,2540.0,260.0,507.0,750.0


In [16]:
result_df.to_csv(f'{data_name}-result/laftr-{epochs}.csv')